In [ ]:
import pandas as pd
import math
from sklearn import metrics
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt
from utils import bootstrap
from 
from tqdm import tqdm
from matplotlib import rc

def bootstrap_ppv_f1_recall(x,y):
    y_total,yhat_total = x,y
    ppv_list = []
    f1_score_list = []
    recall_list = []
    
    # bootstrap for confidence interval
    for i in tqdm(range(0,10000)):
        choices = np.random.choice(range(0,len(yhat_total)),int(len(yhat_total)/2))
        ground = y_total[choices]
        ground.index = range(0,len(ground))
        preds = yhat_total[choices]
        preds.index = range(0,len(preds))
        ppv_list.append(metrics.precision_score(ground, preds))
        f1_score_list.append(metrics.f1_score(ground, preds))
        recall_list.append(metrics.recall_score(ground, preds))
        
    lower_point_ppv = round(np.percentile(ppv_list,2.5),3)
    higher_point_ppv = round(np.percentile(ppv_list,97.5),3)
    
    lower_point_f1 = round(np.percentile(f1_score_list,2.5),3)
    higher_point_f1 = round(np.percentile(f1_score_list,97.5),3)
    
    lower_point_recall = round(np.percentile(recall_list,2.5),3)
    higher_point_recall = round(np.percentile(recall_list,97.5),3)

    
    ppv_preds = [lower_point_ppv, higher_point_ppv]
    f1_preds = [lower_point_f1, higher_point_f1]
    recall_preds = [lower_point_recall, higher_point_recall]
    
    print('PPV is ' + str(ppv_preds))
    print('F1 is ' + str(f1_preds))
    print('Recall is ' + str(recall_preds))

test_predictions = pd.read_csv()
test_predictions = test_predictions.drop_duplicates('filename')
test_predictions.index = range(0,len(test_predictions))


cols = ['Control_preds','Mild_preds','Moderate_preds','Severe_preds']

for i in cols:
    test_predictions[i] = test_predictions[i].apply(sigmoid)

test_predictions.final_class.value_counts()

test_predictions['predicted'] = test_predictions[cols].idxmax(axis = 1).astype(str).str.slice(stop = -6)

test_predictions.final_class.value_counts()

min_number = pd.DataFrame(test_predictions.final_class.value_counts()).final_class[-1]
ordered_indices = ['Control','Mild','Moderate','Severe']

cm = metrics.confusion_matrix(test_predictions['final_class'], test_predictions['predicted'])

cm = pd.DataFrame(cm, columns = np.sort(test_predictions.final_class.unique()),
                  index = np.sort(test_predictions.final_class.unique()))


fpr_severe, tpr_severe, thresholds = metrics.roc_curve((test_predictions.final_class == 'Severe') * 1, 
                                         test_predictions.Severe_preds)
fpr_mod_severe, tpr_mod_severe, thresholds = metrics.roc_curve((test_predictions.final_class.isin(
    ['Moderate','Severe']) * 1), test_predictions[['Moderate_preds','Severe_preds']].max(axis = 1))

scaling_factor = 1.5

plt.figure(figsize=(8*scaling_factor,8*scaling_factor))

lw = 2*scaling_factor
ls = 'dashed'
stanford_color = 'C0'
cedars_color = 'C2'

## Make severe a solid line
## Make moderate to severe a dashed line

plt.plot(fpr_severe, tpr_severe, linewidth = lw, color = cedars_color, label = ('Severe - AUC: ' + str(round(metrics.auc(fpr_severe, tpr_severe),3))))
plt.plot(fpr_mod_severe, tpr_mod_severe, linestyle = ls, linewidth = lw, color = cedars_color, label = '≥ Moderate - AUC: ' + str(round(metrics.auc(fpr_mod_severe, tpr_mod_severe),4))[:-1])
plt.xlabel('1- Specificity', fontsize = 20*scaling_factor, rotation = 0, labelpad=10)
plt.ylabel('Sensitivity', fontsize = 20*scaling_factor, rotation = 90, labelpad=15)
# plt.title('MR Detection - Severity and Institution',fontsize = 20)
plt.xticks(fontsize = 17*scaling_factor)
plt.yticks(fontsize = 17*scaling_factor)

legend = plt.legend(title = "Severity and Institution", fontsize = 15*1.5)
plt.setp(legend.get_title(),fontsize= 17*1.5)
# ax.set_ylabel('abc', rotation=0, fontsize=20, labelpad=20)


# plt.plot(, linestyle = 'dashed', color = 'red')
# plt.plot(, linestyle = 'dashed', color = 'red')

In [ ]:

cols = ['Control_preds','Mild_preds','Moderate_preds','Severe_preds']

for i in cols:
    test_predictions[i] = test_predictions[i].apply(sigmoid)

test_predictions.final_class.value_counts()

test_predictions['predicted'] = test_predictions[cols].idxmax(axis = 1).astype(str).str.slice(stop = -6)

test_predictions.final_class.value_counts()

min_number = pd.DataFrame(test_predictions.final_class.value_counts()).final_class[-1]
ordered_indices = ['Control','Mild','Moderate','Severe']

# control_index = test_predictions[test_predictions.final_class.isin(['Control'])].sample(n = min_number).index
# mild_mr_index = test_predictions[test_predictions.final_class.isin(['Mild'])].sample(n = min_number).index
# moderate_mr_index = test_predictions[test_predictions.final_class.isin(['Moderate'])].sample(n = min_number).index
# severe_mr_index = test_predictions[test_predictions.final_class.isin(['Severe'])].index
# indices = list(control_index) + list(mild_mr_index) + list(moderate_mr_index) + list(severe_mr_index)
# test_predictions = test_predictions[test_predictions.index.isin(indices)]

cm = metrics.confusion_matrix(test_predictions['final_class'], test_predictions['predicted'])

cm = pd.DataFrame(cm, columns = np.sort(test_predictions.final_class.unique()),
                  index = np.sort(test_predictions.final_class.unique()))
# cm = cm[ordered_indices].loc[ordered_indices]
# cm_original = cm

plt.figure(figsize=(10,10))
sns.set(font_scale=2)

# cm_try = cm/cm.sum(axis = 1)
cm_try = cm.copy()
cm_try.loc['Control'] = cm.loc['Control']/cm.loc['Control'].sum()
cm_try.loc['Mild'] = cm.loc['Mild']/cm.loc['Mild'].sum()
cm_try.loc['Moderate'] = cm.loc['Moderate']/cm.loc['Moderate'].sum()
cm_try.loc['Severe'] = cm.loc['Severe']/cm.loc['Severe'].sum()

cm_try = cm_try.rename(columns = {'Control':'None'}, index = {'Control':'None'})

res = sns.heatmap(cm_try, annot=cm.to_numpy(), cmap='Greens', linewidths = 5, linecolor='black',fmt=',d',
           vmin = 0, vmax = cm_try.to_numpy().max(), cbar = False, square = True)   
    
# Drawing the frame 
res.axhline(y = 0, color='k',linewidth = 10) 
res.axhline(y = cm.shape[1], color = 'k', 
            linewidth = 10) 
  
res.axvline(x = 0, color = 'k', 
            linewidth = 10) 
  
res.axvline(x = cm.shape[0],  
            color = 'k', linewidth = 10) 
  
# show plot 
plt.title('Mitral Regurgitation Classification - CSMC\n', fontsize = 25, fontweight = 'bold')
plt.ylabel('Actual  ', fontsize = 23, rotation = 0, fontweight = 'bold')
plt.xlabel('\nPredicted', fontsize = 23, fontweight = 'bold')
plt.xticks(fontsize = 20)
plt.yticks(fontsize = 20, rotation = 0)

# for _, spine in res.spines.items(): 
#     spine.set_visible(True) 
#     spine.set_linewidth(5)
    
plt.show()


test_predictions['severe_binary'] = (test_predictions['final_class'].isin(['Severe'])*1)
test_predictions['severe_binary_pred'] = (test_predictions['predicted'].isin(['Severe'])*1)
test_predictions['mod_severe_binary'] = (test_predictions['final_class'].isin(['Moderate','Severe'])*1)
test_predictions['mod_severe_binary_pred'] = (test_predictions['predicted'].isin(['Moderate','Severe'])*1)
test_predictions['control_mild_binary'] = (test_predictions['final_class'].isin(['Control','Mild'])*1)
test_predictions['control_mild_binary_pred'] = (test_predictions['predicted'].isin(['Control','Mild'])*1)
test_predictions['moderate_binary'] = (test_predictions['final_class'].isin(['Moderate'])*1)
test_predictions['moderate_binary_pred'] = (test_predictions['predicted'].isin(['Moderate'])*1)

test_predictions['not_severe_binary'] = (~test_predictions['final_class'].isin(['Severe'])*1)
test_predictions['not_severe_binary_pred'] = (~test_predictions['predicted'].isin(['Severe'])*1)

test_predictions['Mod_Severe_preds'] = test_predictions[
    ['Moderate_preds','Severe_preds']].max(axis = 1, skipna = True)



In [ ]:
### Severe

print('Severe PPV is ' + str(round(metrics.precision_score(test_predictions['severe_binary'], test_predictions['severe_binary_pred'],
                       labels = ['Severe']),3)))

print('Severe NPV is ' + str(round(metrics.precision_score(test_predictions['not_severe_binary'], test_predictions['not_severe_binary_pred'],
                       labels = ['Severe']),3)))

print('Severe Recall is ' + str(round(metrics.recall_score(test_predictions['severe_binary'], test_predictions['severe_binary_pred'],
                       labels = ['Severe']),3)))

print('Severe F1-Score is ' + str(round(metrics.f1_score(test_predictions['severe_binary'], test_predictions['severe_binary_pred'],
                       labels = ['Severe']),3)) + '\n')



### Moderate/Severe

print('Moderate/Severe PPV is ' + str(round(metrics.precision_score(test_predictions['mod_severe_binary'], test_predictions['mod_severe_binary_pred'],
                       labels = ['Moderate/Severe']),3)))


print('Moderate/Severe NPV is ' + str(round(metrics.precision_score(test_predictions['control_mild_binary'], test_predictions['control_mild_binary_pred'],
                       labels = ['Moderate/Severe']),3)))

print('Moderate/Severe Recall is ' + str(round(metrics.recall_score(test_predictions['mod_severe_binary'], test_predictions['mod_severe_binary_pred'],
                       labels = ['Moderate/Severe']),3)))

print('Moderate/Severe F1-Score is ' + str(round(metrics.f1_score(test_predictions['mod_severe_binary'], test_predictions['mod_severe_binary_pred'],
                       labels = ['Moderate/Severe']),3)))